In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, AveragePooling2D, concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report, precision_score

# Set the paths to your train, test, and validation data
train_dir = '/content/drive/MyDrive/New folder (2)/train'
test_dir = '/content/drive/MyDrive/test'
validation_dir = '/content/drive/MyDrive/New folder (2)/validation'
#hyperparameters
epochs = 21
batch_size = 16
learning_rate = 0.0001


# Preprocessing the images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Custom CNN model
cnn_input = Input(shape=(224, 224, 3))
x = Conv2D(64, (3, 3), activation='relu')(cnn_input)
x = MaxPooling2D((2, 2))(x)
# Add more convolutional and pooling layers as needed for your custom CNN
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
cnn_output = Dense(8, activation='softmax')(x)  # Change 14 to the number of classes for the CNN model
cnn_model = Model(inputs=cnn_input, outputs=cnn_output)

# GoogleNet (Inception) model
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

inception_output = inception_model(cnn_input)  # Output from the Inception model
x = AveragePooling2D(pool_size=(5,5))(inception_output)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
inception_output = Dense(14, activation='softmax')(x)  # Change 14 to the number of classes for the Inception model
inception_model = Model(inputs=cnn_input, outputs=inception_output)

# Hybrid model
x = concatenate([cnn_output, inception_output], axis=-1)  # Concatenate the CNN and Inception outputs
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
output_layer = Dense(14, activation='softmax')(x)  # Change 8 to the number of classes for the hybrid model
model = Model(inputs=cnn_input, outputs=output_layer)

optimizer = RMSprop(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)






# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size
)

# Plot training and validation loss graphs
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, epochs + 1)

plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle =False
)
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.n // batch_size)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Saving the model
model.save('/content/drive/MyDrive/model_save/hybrid_model.h5')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
model = tf.keras.models.load_model('/content/drive/MyDrive/model_save/hybrid_model.h5')
# Load and preprocess the test image
img_path = '/content/drive/MyDrive/New folder (2)/validation/Alaskan Malamute dog/Image_27.jpg_gaussian_blur.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # Normalizing the image

# Making prediction
prediction = model.predict(img)
predicted_class_index = np.argmax(prediction)
predicted_probability = np.max(prediction)
# give the class names
class_names = ['Affenhuhua_dog', 'Afghan_Hound dog', 'Akita_DOG','Alaskan malamute_dog','American_Bulldog_dog','Auggie_dog','Beagle_dog','Belgian Tervuren_dog','Bichon Frise_dog','Bocker_dog','Borzoi_dog','Boxer_dog','Bugg_dog','Bulldog_dog']
predicted_class_name = class_names[predicted_class_index]
percentage = predicted_probability * 100
# Printing the predicted class name and percentage
print(f"Predicted Class: {predicted_class_name} ({percentage:.2f}%)")
img = mpimg.imread(img_path)
# Display the image
plt.imshow(img)
plt.axis('on')
plt.show()